### Ejercicio 
ps(1) permite ver los procesos del sistema y su estado. Estudiar la página de 
manual y determinar las opciones necesarias para: 

Mostrar todos los procesos del usuario actual en formato extendido. (Nota: usar la variable de entorno USER). 

In [ ]:
$ ps -U $USSER -u

Mostrar los procesos del sistema, incluyendo el identificador del proceso, el identificador 
del grupo de procesos, el identificador de sesión, el estado y el comando con todos sus 
argumentos.

In [ ]:
$ ps -ahl

#### Observar el identificador de proceso, grupo de procesos y sesión de los procesos. ¿Qué identificadores comparten la shell y los programas que se ejecutan en ella (usar la opción -H para identificar fácilmente la relación entre procesos)? 

Ejecutando el comando $ ps -ahl

> Identificador del proceso (PID).

> Identificador del proceso padre (PPID).

- Los procesos hijo comparten el identificador del padre (ppid)
- Comparten tambien la prioridad del proceso (Pri) (la prioridad asignada por el kernel)
- Comparten el nice level (ni) (numero modificable capaz de moficar la prioridad asignada por el kernel)


#### ¿Cuál es el identificador de grupo de procesos cuando se crea un nuevo proceso? Usar el comando sleep para poder verlo fácilmente en la salida del comando ps. 

Todos los procesos comparten el identificador del padre (ppid)

### Ejercicio 2. Escribir un programa que muestre los identificadores del proceso (PID, PPID, PGID y SID), el identificador del usuario y grupo, y su directorio de trabajo actual. 

In [ ]:
#include <stdio.h>
#include <stdlib.h>
#include <unistd.h>

int main(int argc, char argv)
{
    int pid = getpid();
    int ppid = getppid();
    int pgid = getpgid(pid);
    int sid = getsid(pid);

    printf("PID: %i, PPID: %i, PGID: %i, SID: %i\n", pid, ppid, pgid, sid);

    return 0;
}

### Ejercicio 3. Considera la siguiente orden de la línea de comandos: 
> echo "12345" > /proc/$$/fd/1 

Lo que estas haciendo es escribir en la salida estandar(fd/1) del proceso actual($$) -> stdout: escribe en consola 12345

### Ejercicio 4. Consulta el tipo de fichero y contenido de los siguientes ficheros del proceso de la shell actual (/proc/$$) completa la siguiente tabla: 

### Ejercicio 5
Escribir un programa que cree un proceso hijo con las siguientes características:
- El programa recibirá dos argumentos en la forma: 
> ./ejercicio5 <segundos_padre> <segundos_hijo>
- El hijo creará su propia sesión, imprimirá sus identificadores (como en el Ejercicio 2), esperará <segundos hijo> (segundo argumento) con la llamada sleep(3); y terminará.
- El padre imprimirá sus identificadores (Ejercicio 2), esperará <segundos padre> (primer argumento) con la llamada sleep; y terminará.

In [ ]:
int main() {
    pid_t pid;

    pid = fork();

    switch (pid) {
    case -1:
        perror("fork");
        exit(1);
    case 0:
        sleep(argv[2]);
        printf("Hijo: %i (padre: %i)\n", getpid(), getppid());
        break;
    default:
        sleep(argv[1]);
        printf("Padre: %i (hijo: %i)\n", getpid(), pid);
        break;
    }

    return 0;
}

### Ejercicio 9:
Escribir un programa que ejecute otro programa (ejecutable y argumentos) que se pasará como argumento. El programa creará un proceso hijo que ejecutará el programa dado en el argumento con la función execvp(3). El proceso padre esperará que termine el hijo e imprimirá su código de salida. Ejemplos de ejecución:


In [ ]:
// Queremos ejecutar ./9 ls -l -a

int main() {
    pid_t pid;

    pid = fork();

    switch (pid) {
    case -1: // Error
        perror("fork");
        exit(1);
    case 0: // Hijo
        printf("Hijo: %i (padre: %i)\n", getpid(), getppid());
        //  &argv[1] direccion del primer argumento (ls).
        execvp(argv[1], &argv[1]);
        break;
    default: // Padre
        printf("Padre: %i (hijo: %i)\n", getpid(), pid);
        break;
    }

    return 0;
}

### Ejercicio 11 y 12
Escribir un programa que genere un fichero con 5 procesos concurrentes con la siguiente estructura:

![image.png](attachment:image.png)

El proceso padre creará el fichero de salida output.txt y escribirá el primer segmento de 5 ceros (00000). A continuación creará 5 hijos y esperará a que termine cada uno mostrando el PID y número de hijo. Cada hijo realiza las siguientes acciones:
- Abrirá el archivo y desplazará el puntero de escritura el offset correspondiente.
- Escribirá la secuencia correspondiente (“11111”, “22222”...). Esta parte puede implementarse de dos formas:
    - Creando una cadena con la secuencia correspondiente, con sprintf(3); y escribiendo la cadena en el archivo.
    - Definiendo un array con cada posición inicializada al carácter correspondiente al hijo (char c = ‘0’ + index) ; y escribiendo el array en el archivo


In [ ]:
#define NUM 5

void imprimir_indice(int fd, int indice){
    char buffer[NuM];
    for(int i=0; i<NUM; i++){
        buffer[i] = '0' + indice; // Desplazamiento del puntero
    }

    write(fd, buffer, NUM*sizeof(char));

    /*
    char buffer[NUM+1];
    
    
    */
}

int main(int argc, char *argv[]) {
    pid_t pid;
    int rc;

    int fd = open("archivo.txt", 
        O_WRONLY | O_CREATE | O_TRUNC, 
        0660);
    
    imprimir_indice(fd, 0); // Padre

    close(fd);

    for(int indice = 1; indice <= 6; indice++){
        pid_t pid = fork();

        // Solo el padre crea nuevos procesos
        if (pid != 0) {
            continue;
        }
        // El padre no entra aqui
        // cada hijo tiene su archivo
        fd = open("archivo.txt", O_WRONLY);
        // Como hijo, nos posicionamos en el lugar correcto
        lseek(fd, indice * NUM, SEEK_SET);
        srandom(indice);
        int sec = random() % 2;
        sleep(sec);
        imprimir_indice(fd, indice);

        close(fd);
        return 0;
    }

    // Esperamos a los hijos
    for(int i=0; i<=5; i++){
        int s;
        int rc = waitpid(0, &s, 0);
        // Obtener el PID del hijo que ha terminado
        if (WIFEXITED(s)) {
            printf("Hijo [%i]finalizado con codigo %d\n", rc, WEXITSTATUS(s));
        }
    }
    return 0;
}